<a href="https://colab.research.google.com/github/nidheesh-p/AI-Learning/blob/master/Customer_Support_Assistant_(Multi_Agent_with_LangGraph).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

📝 Case Study: Customer Support Assistant (Multi-Agent with LangGraph)
🎯 Goal

Simulate a customer support system with 3 agents:

Router Agent → Understands the query and routes it.

FAQ Agent → Answers product/policy questions.

Escalation Agent → For anything it can’t handle.

In [1]:
!pip install langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.6/207.6 kB 8.9 MB/s eta 0:00:00


In [2]:
from typing import TypedDict

class AgentState(TypedDict):
    query: str
    category: str
    answer: str

In [3]:
### Define Agents

# Router Agent
def router_agent(state: AgentState) -> AgentState:
    q = state["query"].lower()
    if "refund" in q or "return" in q:
        state["category"] = "faq"
    else:
        state["category"] = "escalate"
    return state

# FAQ Agent
def faq_agent(state: AgentState) -> AgentState:
    faq_db = {
        "refund": "Our refund policy allows returns within 30 days of purchase.",
        "return": "You can return items via our online portal or any store location."
    }
    for k, v in faq_db.items():
        if k in state["query"].lower():
            state["answer"] = v
            return state
    state["answer"] = "Sorry, I don’t know the answer."
    return state

# Escalation Agent
def escalation_agent(state: AgentState) -> AgentState:
    state["answer"] = "This query needs a human support agent. Escalating..."
    return state

In [4]:
### Build Workflow

from langgraph.graph import StateGraph, END

workflow = StateGraph(AgentState)

# Add nodes
workflow.add_node("router", router_agent)
workflow.add_node("faq", faq_agent)
workflow.add_node("escalate", escalation_agent)

# Entry point
workflow.set_entry_point("router")

# Conditional edges
def route_from_router(state: AgentState):
    if state["category"] == "faq":
        return "faq"
    else:
        return "escalate"

workflow.add_conditional_edges("router", route_from_router, {
    "faq": "faq",
    "escalate": "escalate"
})

# Both FAQ and Escalation go to END
workflow.add_edge("faq", END)
workflow.add_edge("escalate", END)

app = workflow.compile()


In [5]:
### Run Demo

queries = [
    "How do I get a refund?",
    "Can I return a product?",
    "What are your store timings?"
]

for q in queries:
    print("\n==============================")
    print("User Query:", q)
    result = app.invoke({"query": q})
    print("Category:", result["category"])
    print("Answer:", result["answer"])



User Query: How do I get a refund?
Category: faq
Answer: Our refund policy allows returns within 30 days of purchase.

User Query: Can I return a product?
Category: faq
Answer: You can return items via our online portal or any store location.

User Query: What are your store timings?
Category: escalate
Answer: This query needs a human support agent. Escalating...
